In [1]:
import pymysql
import time
import json
import requests

In [2]:
host = "localhost"
port = 3309
database = "insup"
username = "root"
password = "amadeus21!"

In [3]:
conn = pymysql.connect(
    host=host,
    user=username,
    password=password,
    database=database,
    port=port
)

In [4]:
response = requests.post(f"https://www.dhlottery.co.kr/store.do?method=sellerInfo645Result&searchType=3&nowPage=1&sltSIDO2=서울&sltGUGUN2=&rtlrSttus=000")

response.json()['arr']
              
              

[{'BPLCLOCPLC3': '중계동',
  'BPLCLOCPLC2': '노원구',
  'BPLCLOCPLC1': '서울',
  'FIRMNM': 'CU&&#35;40;중계위성점&&#35;41;',
  'DEALSPEETO': 'N',
  'LONGITUDE': 127.076265,
  'BPLCLOCPLC4': None,
  'RECORDNO': 1,
  'BPLCLOCPLCDTLADRES': '360-13',
  'RTLRID': '21100494',
  'RTLRSTRTELNO': '02-937-7009',
  'DEAL645': '1',
  'BPLCDORODTLADRES': '서울 노원구 한글비석로 231 ',
  'DEAL520': 'N',
  'LATITUDE': 37.648192},
 {'BPLCLOCPLC3': '창동',
  'BPLCLOCPLC2': '도봉구',
  'BPLCLOCPLC1': '서울',
  'FIRMNM': 'CU&&#35;40;창동미소점&&#35;41;',
  'DEALSPEETO': 'N',
  'LONGITUDE': 127.045202,
  'BPLCLOCPLC4': None,
  'RECORDNO': 2,
  'BPLCLOCPLCDTLADRES': '254-23',
  'RTLRID': '21120032',
  'RTLRSTRTELNO': '02-904-0133',
  'DEAL645': '1',
  'BPLCDORODTLADRES': '서울 도봉구 노해로63가길 14 ',
  'DEAL520': 'N',
  'LATITUDE': 37.653323},
 {'BPLCLOCPLC3': '명륜1가',
  'BPLCLOCPLC2': '종로구',
  'BPLCLOCPLC1': '서울',
  'FIRMNM': 'GS25&&#35;40;명륜성대점&&#35;41;',
  'DEALSPEETO': 'N',
  'LONGITUDE': 126.996553,
  'BPLCLOCPLC4': None,
  'RECORDNO': 3,
  'BP

In [5]:
def brace_to_single_quote(word):
    return "\'" + word + "\'"

In [18]:
for si in ["서울","경기","부산","대구","인천","대전","울산","강원","충북","충남","광주","전북","전남","경북","경남","제주","세종"]:
    response_for_total = requests.post(f"https://www.dhlottery.co.kr/store.do?method=sellerInfo645Result&searchType=3&nowPage=1&sltSIDO2={si}&sltGUGUN2=&rtlrSttus=000")
    total_page = response_for_total.json()['totalPage']    
    for nowPage in range(1, total_page+1):
        response = requests.post(f"https://www.dhlottery.co.kr/store.do?method=sellerInfo645Result&searchType=3&nowPage=1&sltSIDO2={si}&sltGUGUN2=&rtlrSttus=000")
        store_list = response.json()['arr']
        
        for store in store_list:
            cursor = conn.cursor()
            
            si = brace_to_single_quote(store['BPLCLOCPLC1'].strip())
            gu = brace_to_single_quote(store['BPLCLOCPLC2'].strip())
            dong = brace_to_single_quote(store['BPLCLOCPLC3'].strip())
            store_name = brace_to_single_quote(store['FIRMNM'].strip())
            longitude = store['LONGITUDE']
            latitude = store['LATITUDE']
            address = brace_to_single_quote(store['BPLCDORODTLADRES'].strip())
            phone_no = brace_to_single_quote(store['RTLRSTRTELNO'].strip())
            retailer_id = int(store['RTLRID'].strip())
            active = 1 if store['DEAL645'] == "1" else 0
            
            query = f'insert into lotto_store(store_name, location, si, gu, dong, address, active, retailer_id)\
            VALUES ({store_name}, POINT({latitude}, {longitude}), {si}, {gu}, {dong}, {address}, {active}, {retailer_id})'

            cursor.execute(query)

            conn.commit()

            cursor.close()
            
        time.sleep(2)
        print(f"{si} {str(nowPage)}페이지 완료")

        

OperationalError: (1364, "Field 'id' doesn't have a default value")

In [19]:
conn.close()